# MMM Analysis Notebook
This notebook contains the main analysis for the Marketing Mix Modeling project.

In [3]:
# Cell 1: Setup and Imports
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Custom imports
from data_processing import load_and_validate_data, clean_media_data, clean_control_variables, create_dataset_splits
from feature_engineering import process_media_channels, create_seasonality_features, prepare_model_features
from mediation_analysis import analyze_causal_paths, compute_mediation_effects
from modeling import create_elasticnet_model, train_model_with_timeseries_cv, evaluate_model_performance, extract_model_coefficients
from evaluation import create_prediction_plots, plot_residual_analysis, calculate_marketing_roas, analyze_price_sensitivity, analyze_promotion_impact

# Set random seed for reproducibility
np.random.seed(42)

# Create results directories
os.makedirs('../results/models', exist_ok=True)
os.makedirs('../results/figures', exist_ok=True)
os.makedirs('../results/tables', exist_ok=True)

print("✅ Setup complete - All modules imported successfully")

ModuleNotFoundError: No module named 'pandas'

In [12]:
# Cell 2: Data Loading and Initial Exploration
# Load the dataset
data_path = "../data/weekly_data.csv"
df_raw = load_and_validate_data(data_path, date_col='week')

print("\n=== Dataset Overview ===")
print(f"Shape: {df_raw.shape}")
print(f"Date range: {df_raw['week'].min()} to {df_raw['week'].max()}")
print(f"Columns: {list(df_raw.columns)}")

# Display basic statistics
print("\n=== Revenue Statistics ===")
print(df_raw['revenue'].describe())

# Check for missing values
print("\n=== Missing Values ===")
print(df_raw.isnull().sum())

print("✅ Data loaded and validated successfully")

NameError: name 'load_and_validate_data' is not defined

In [ ]:
# Cell 3: Data Cleaning and Preprocessing
# Define feature categories
media_channels = ['facebook_spend', 'google_spend', 'tiktok_spend', 'instagram_spend', 'snapchat_spend']
control_variables = ['social_followers', 'average_price', 'promotions', 'emails_send', 'sms_send']

# Clean media data
df_cleaned = clean_media_data(df_raw, media_channels)

# Clean control variables
df_cleaned = clean_control_variables(df_cleaned, control_variables)

print("✅ Data cleaning completed")

In [ ]:
# Cell 4: Feature Engineering
# Process media channels with adstock and saturation
df_processed = process_media_channels(df_cleaned, media_channels, decay_rate=0.6, max_lags=8)

# Add trend and seasonality features
df_processed = create_seasonality_features(df_processed, date_col='week')

# Prepare feature lists for modeling
social_media_features, all_media_features, control_features = prepare_model_features(
    df_processed, media_channels, 
    ['average_price', 'promotions', 'emails_send', 'sms_send', 'social_followers', 
     'trend', 'week_of_year', 'sin_yearly', 'cos_yearly', 'sin_biannual']
)

print(f"✅ Feature engineering completed")
print(f"Social media features: {len(social_media_features)}")
print(f"Control features: {len(control_features)}")
print(f"Total processed features: {len(social_media_features + control_features) + 1}")  # +1 for Google residual

In [ ]:
# Cell 5: Exploratory Data Analysis
# Create visualization of revenue and media spend trends
plt.figure(figsize=(15, 10))

# Revenue and media spend over time
plt.subplot(2, 2, 1)
plt.plot(df_processed['week'], df_processed['revenue'], label="Revenue", color="black", linewidth=2)
for channel in media_channels:
    plt.plot(df_processed['week'], df_processed[channel], label=channel, alpha=0.7)
plt.legend()
plt.title("Revenue & Media Spend Over Time")
plt.xticks(rotation=45)

# Correlation heatmap
plt.subplot(2, 2, 2)
correlation_data = df_processed[media_channels + ['revenue']].corr()
sns.heatmap(correlation_data, annot=True, cmap="RdBu_r", center=0, 
            square=True, cbar_kws={"shrink": 0.8})
plt.title("Media Channels Correlation Matrix")

# Revenue distribution
plt.subplot(2, 2, 3)
plt.hist(df_processed['revenue'], bins=30, alpha=0.7, edgecolor='black')
plt.title("Revenue Distribution")
plt.xlabel("Revenue")
plt.ylabel("Frequency")

# Seasonality pattern
plt.subplot(2, 2, 4)
monthly_revenue = df_processed.groupby(df_processed['week'].dt.month)['revenue'].mean()
plt.bar(monthly_revenue.index, monthly_revenue.values, alpha=0.7)
plt.title("Average Revenue by Month")
plt.xlabel("Month")
plt.ylabel("Average Revenue")

plt.tight_layout()
plt.savefig('../results/figures/eda_overview.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Exploratory data analysis completed")

In [ ]:
# Cell 6: Mediation Analysis - Stage 1
# Perform mediation analysis treating Google as mediator
print("=== MEDIATION ANALYSIS ===")
print("Assumption: Social media channels → Google spend → Revenue")

mediation_results = analyze_causal_paths(
    df_processed, 
    social_media_features, 
    mediator_col='google_spend',
    target_col='revenue',
    control_features=control_features
)

# Extract results
stage1_model = mediation_results['stage1_model']
df_with_residuals = mediation_results['processed_data']
stage2_features = mediation_results['stage2_features']

print(f"✅ Stage 1 completed - Google spend model R²: {stage1_model.rsquared:.3f}")
print(f"✅ Stage 2 feature set prepared: {len(stage2_features)} features")

In [ ]:
# Cell 7: Model Training - Stage 2
# Prepare data for Stage 2 modeling
X_features = df_with_residuals[stage2_features]
y_target = df_with_residuals['revenue']

print(f"Training data shape: X={X_features.shape}, y={y_target.shape}")

# Create and train ElasticNet model with time series CV
elasticnet_model = create_elasticnet_model(cv_folds=5, l1_ratios=[0.1, 0.5, 0.9])

trained_model, oof_predictions, oof_indices = train_model_with_timeseries_cv(
    elasticnet_model, X_features.values, y_target.values, n_splits=5
)

# Evaluate out-of-fold performance
oof_metrics = evaluate_model_performance(
    y_target.values[oof_indices], oof_predictions[oof_indices], 
    model_name="ElasticNet (Out-of-Fold)"
)

print("✅ Stage 2 model training completed")

In [ ]:
# Cell 8: Model Evaluation and Diagnostics
# Get final predictions on full dataset
final_predictions = trained_model.predict(X_features.values)
residuals = y_target.values - final_predictions

# Evaluate full model performance
full_metrics = evaluate_model_performance(
    y_target.values, final_predictions, model_name="ElasticNet (Full Dataset)"
)

# Extract model coefficients
model_coefficients = extract_model_coefficients(trained_model, X_features.columns.tolist())

print("\n=== TOP 10 MODEL COEFFICIENTS ===")
print(model_coefficients.head(10))

# Create prediction plots
create_prediction_plots(
    df_with_residuals['week'], y_target.values, final_predictions,
    save_path='../results/figures/model_predictions.png',
    title="ElasticNet Model Performance"
)

# Create residual analysis plots
plot_residual_analysis(
    df_with_residuals['week'], residuals,
    save_path='../results/figures/residual_analysis.png'
)

print("✅ Model evaluation and diagnostics completed")

In [ ]:
# Cell 9: Business Insights - ROAS Analysis
# Calculate Marketing ROAS for each channel
roas_results = calculate_marketing_roas(
    df_with_residuals, trained_model, stage2_features, 
    media_channels, spend_increase_pct=0.05
)

print("\n=== MARKETING ROAS ANALYSIS ===")
print(roas_results[['channel', 'marginal_roas']].sort_values('marginal_roas', ascending=False))

# Plot ROAS comparison
plt.figure(figsize=(10, 6))
roas_clean = roas_results['channel'].str.replace('_spend', '').str.title()
bars = plt.bar(roas_clean, roas_results['marginal_roas'], 
               color=['red' if x < 1 else 'green' for x in roas_results['marginal_roas']], 
               alpha=0.7)

plt.axhline(y=1, color='black', linestyle='--', alpha=0.7, label='Break-even ROAS')
plt.title('Marginal ROAS by Marketing Channel\n(5% Spend Increase Scenario)', fontsize=14)
plt.ylabel('Revenue per $1 Spent')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, roas_results['marginal_roas']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
             f'${value:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig('../results/figures/roas_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ ROAS analysis completed")

In [ ]:
# Cell 10: Price Sensitivity Analysis
# Analyze price elasticity
price_sensitivity = analyze_price_sensitivity(
    df_with_residuals, trained_model, stage2_features,
    price_col='average_price', price_changes=[-0.1, -0.05, 0.05, 0.1],
)

print("\n=== PRICE SENSITIVITY ANALYSIS ===")
print(price_sensitivity)

# Plot price sensitivity
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.plot(price_sensitivity['price_change_pct'] * 100, 
         price_sensitivity['revenue_change_pct'] * 100, 
         marker='o', linewidth=2, markersize=8)
plt.xlabel('Price Change (%)')
plt.ylabel('Revenue Change (%)')
plt.title('Price vs Revenue Relationship')
plt.grid(True, alpha=0.3)
plt.axhline(0, color='black', linestyle='-', alpha=0.5)
plt.axvline(0, color='black', linestyle='-', alpha=0.5)

plt.subplot(1, 2, 2)
plt.bar(range(len(price_sensitivity)), price_sensitivity['price_elasticity'], 
        alpha=0.7, color='orange')
plt.xlabel('Scenario')
plt.ylabel('Price Elasticity')
plt.title('Price Elasticity by Scenario')
plt.xticks(range(len(price_sensitivity)), 
           [f"{x:.0%}" for x in price_sensitivity['price_change_pct']])
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/price_sensitivity.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Price sensitivity analysis completed")

In [ ]:
# Cell 11: Promotion Impact Analysis
# Analyze promotion effectiveness
promotion_impact = analyze_promotion_impact(
    df_with_residuals, trained_model, stage2_features, promotion_col='promotions'
)

print("\n=== PROMOTION IMPACT ANALYSIS ===")
print(f"Revenue without promotions: ${promotion_impact['revenue_no_promotion']:.2f}")
print(f"Revenue with promotions: ${promotion_impact['revenue_with_promotion']:.2f}")
print(f"Promotion lift: {promotion_impact['promotion_lift_pct']:.2%}")
print(f"Incremental revenue: ${promotion_impact['incremental_revenue']:.2f}")

# Visualize promotion impact
plt.figure(figsize=(8, 6))
scenarios = ['No Promotions', 'With Promotions']
revenues = [promotion_impact['revenue_no_promotion'], promotion_impact['revenue_with_promotion']]
colors = ['lightcoral', 'lightgreen']

bars = plt.bar(scenarios, revenues, color=colors, alpha=0.7, edgecolor='black')
plt.title(f"Promotion Impact Analysis\nLift: {promotion_impact['promotion_lift_pct']:.1%}", fontsize=14)
plt.ylabel('Average Revenue')

# Add value labels
for bar, value in zip(bars, revenues):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(revenues)*0.01, 
             f'${value:.0f}', ha='center', va='bottom')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../results/figures/promotion_impact.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Promotion impact analysis completed")

In [ ]:
# Cell 12: Save All Results
# Save model artifacts
import joblib

# Save trained model
joblib.dump(trained_model, '../results/models/elasticnet_mmm_model.pkl')

# Save processed dataset
df_with_residuals.to_csv('../results/tables/processed_dataset.csv', index=False)

# Save model coefficients
model_coefficients.to_csv('../results/tables/model_coefficients.csv', index=False)

# Save business insights
roas_results.to_csv('../results/tables/roas_analysis.csv', index=False)
price_sensitivity.to_csv('../results/tables/price_sensitivity.csv', index=False)

# Save promotion results
promotion_df = pd.DataFrame([promotion_impact])
promotion_df.to_csv('../results/tables/promotion_impact.csv', index=False)

# Save model performance metrics
performance_metrics = pd.DataFrame([full_metrics])
performance_metrics.to_csv('../results/tables/model_performance.csv', index=False)

# Create predictions table
predictions_table = pd.DataFrame({
    'week': df_with_residuals['week'],
    'actual_revenue': y_target.values,
    'predicted_revenue': final_predictions,
    'residuals': residuals,
    'abs_error': np.abs(residuals),
    'pct_error': np.abs(residuals) / y_target.values * 100
})
predictions_table.to_csv('../results/tables/model_predictions.csv', index=False)

print("✅ All results saved successfully")

In [ ]:
# Cell 13: Mediation Effects Analysis
# Calculate mediation effects using Stage 1 and Stage 2 results
# Note: This requires a custom implementation since we used ML for Stage 2

# For demonstration, we'll calculate approximate mediation effects
mediation_effects_data = []

for social_feature in social_media_features:
    # Get coefficient from Stage 2 model
    stage2_coef = model_coefficients[model_coefficients['feature'] == social_feature]
    direct_effect = stage2_coef['coefficient'].iloc[0] if not stage2_coef.empty else 0
    
    # Get coefficient from Stage 1 model (social -> google)
    stage1_coef = stage1_model.params.get(social_feature, 0)
    
    # Google residual coefficient from Stage 2
    google_coef = model_coefficients[model_coefficients['feature'] == 'google_spend_residual']
    google_effect = google_coef['coefficient'].iloc[0] if not google_coef.empty else 0
    
    # Calculate indirect effect (social -> google -> revenue)
    indirect_effect = stage1_coef * google_effect
    
    # Total effect
    total_effect = direct_effect + indirect_effect
    
    mediation_effects_data.append({
        'channel': social_feature.replace('_transformed', ''),
        'direct_effect': direct_effect,
        'indirect_effect': indirect_effect,
        'total_effect': total_effect,
        'mediation_ratio': indirect_effect / total_effect if total_effect != 0 else 0
    })

mediation_effects_df = pd.DataFrame(mediation_effects_data)

print("\n=== MEDIATION EFFECTS ANALYSIS ===")
print("Direct Effect: Social Media → Revenue")
print("Indirect Effect: Social Media → Google → Revenue")
print()
print(mediation_effects_df)

# Save mediation results
mediation_effects_df.to_csv('../results/tables/mediation_effects.csv', index=False)

# Visualize mediation effects
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Direct vs Indirect effects
x = np.arange(len(mediation_effects_df))
width = 0.35

ax1.bar(x - width/2, mediation_effects_df['direct_effect'], width, 
        label='Direct Effect', alpha=0.7, color='blue')
ax1.bar(x + width/2, mediation_effects_df['indirect_effect'], width,
        label='Indirect Effect (via Google)', alpha=0.7, color='red')

ax1.set_xlabel('Marketing Channel')
ax1.set_ylabel('Effect Size')
ax1.set_title('Direct vs Indirect Effects')
ax1.set_xticks(x)
ax1.set_xticklabels([ch.replace('_spend', '') for ch in mediation_effects_df['channel']], rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Mediation ratios
ax2.bar(range(len(mediation_effects_df)), mediation_effects_df['mediation_ratio'], 
        alpha=0.7, color='green')
ax2.set_xlabel('Marketing Channel')
ax2.set_ylabel('Mediation Ratio')
ax2.set_title('Proportion of Effect Mediated by Google')
ax2.set_xticks(range(len(mediation_effects_df)))
ax2.set_xticklabels([ch.replace('_spend', '') for ch in mediation_effects_df['channel']], rotation=45)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0.5, color='black', linestyle='--', alpha=0.5, label='50% Mediation')
ax2.legend()

plt.tight_layout()
plt.savefig('../results/figures/mediation_effects.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Mediation effects analysis completed")

In [ ]:
# Cell 14: Final Summary Report
from evaluation import create_business_summary_report

# Generate comprehensive business summary
business_summary = create_business_summary_report(
    full_metrics, roas_results, price_sensitivity, 
    promotion_impact, model_coefficients
)

print(business_summary)

# Save summary report
with open('../results/business_summary_report.txt', 'w') as f:
    f.write(business_summary)

# Create final dashboard-style visualization
fig = plt.figure(figsize=(20, 12))

# Model performance
ax1 = plt.subplot(2, 4, 1)
metrics_names = ['R²', 'MAPE']
metrics_values = [full_metrics['r2_score'], full_metrics['mape']]
colors = ['green' if x > 0.8 else 'orange' if x > 0.6 else 'red' for x in [full_metrics['r2_score'], 1-full_metrics['mape']]]
plt.bar(metrics_names, [full_metrics['r2_score'], 1-full_metrics['mape']], color=colors, alpha=0.7)
plt.title('Model Performance')
plt.ylim(0, 1)

# ROAS comparison
ax2 = plt.subplot(2, 4, 2)
roas_channels = roas_results['channel'].str.replace('_spend', '')
plt.bar(roas_channels, roas_results['marginal_roas'], 
        color=['green' if x > 1 else 'red' for x in roas_results['marginal_roas']], alpha=0.7)
plt.title('Channel ROAS')
plt.xticks(rotation=45)
plt.axhline(y=1, color='black', linestyle='--', alpha=0.5)

# Top coefficients
ax3 = plt.subplot(2, 4, 3)
top_coefs = model_coefficients.head(8)
plt.barh(range(len(top_coefs)), top_coefs['coefficient'], 
         color=['blue' if x > 0 else 'red' for x in top_coefs['coefficient']], alpha=0.7)
plt.yticks(range(len(top_coefs)), [f.replace('_', ' ').title()[:15] for f in top_coefs['feature']])
plt.title('Top Model Coefficients')
plt.xlabel('Coefficient Value')

# Price sensitivity
ax4 = plt.subplot(2, 4, 4)
plt.plot(price_sensitivity['price_change_pct'] * 100, 
         price_sensitivity['revenue_change_pct'] * 100, 
         marker='o', linewidth=2, color='orange')
plt.title('Price Elasticity')
plt.xlabel('Price Change (%)')
plt.ylabel('Revenue Change (%)')
plt.grid(True, alpha=0.3)

# Revenue prediction over time
ax5 = plt.subplot(2, 4, (5, 6))
plt.plot(df_with_residuals['week'], y_target.values, label='Actual', color='black', linewidth=2)
plt.plot(df_with_residuals['week'], final_predictions, label='Predicted', color='red', alpha=0.8)
plt.title('Revenue: Actual vs Predicted')
plt.legend()
plt.xticks(rotation=45)

# Residuals
ax6 = plt.subplot(2, 4, 7)
plt.hist(residuals, bins=20, alpha=0.7, color='blue', edgecolor='black')
plt.title('Model Residuals')
plt.xlabel('Residual Value')
plt.ylabel('Frequency')

# Mediation effects
ax7 = plt.subplot(2, 4, 8)
x_pos = np.arange(len(mediation_effects_df))
plt.bar(x_pos - 0.2, mediation_effects_df['direct_effect'], 0.4, 
        label='Direct', alpha=0.7, color='blue')
plt.bar(x_pos + 0.2, mediation_effects_df['indirect_effect'], 0.4,
        label='Indirect', alpha=0.7, color='red')
plt.title('Mediation Effects')
plt.xticks(x_pos, [ch.replace('_spend', '')[:3] for ch in mediation_effects_df['channel']])
plt.legend()

plt.suptitle('Marketing Mix Model - Executive Dashboard', fontsize=16, y=0.98)
plt.tight_layout()
plt.savefig('../results/figures/executive_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Executive dashboard created")
print("\n" + "="*60)
print("🎉 MARKETING MIX MODEL ANALYSIS COMPLETE! 🎉")
print("="*60)
print("\nKey Deliverables:")
print("📊 Trained ElasticNet model with mediation structure")
print("📈 ROAS analysis for all marketing channels")
print("💰 Price elasticity and promotion impact analysis")
print("🔍 Mediation effects (Social → Google → Revenue)")
print("📋 Comprehensive business summary report")
print("\nAll results saved in '../results/' directory")
print("="*60)